# **Face Recognition Using Convolutional Neural Networks**

#### In today's lab, we'll be implementing image classification using transfer learning with two popular CNN architectures using Transfer Learning: VGG16 and ResNet50.

## **Imports**
First, we need to set up our environment and import the necessary libraries.

In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory

## **Data Preparation**

I have uploaded the data onto my github, so we will clone the repository and access the dataset from the cloned local repository.

In [3]:
!git clone https://github.com/shravan-18/Deep-Learning-Lab-Resources.git

Cloning into 'Deep-Learning-Lab-Resources'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 25 (delta 4), reused 22 (delta 4), pack-reused 0
Receiving objects: 100% (25/25), 19.74 MiB | 44.72 MiB/s, done.
Resolving deltas: 100% (4/4), done.


We have to unzip the data since I had uploaded it as a zip file. We'll unzip it into the "/content" directory

In [ ]:
!unzip "/content/Deep-Learning-Lab-Resources/Lab-4/Face Images.zip" -d /content

Define Data Directory and Parameters

In [ ]:
data_dir = '/content/Face Images/Final Training Images'
batch_size = 32
target_size = (224, 224)

Split Data into Training and Validation Sets

In [ ]:
# Define ImageDataGenerators for training and validation
datagen_train = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% of the data will be used for validation
)

datagen_val = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load the training and validation datasets
train_dataset = datagen_train.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

val_dataset = datagen_val.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)

# Display a sample image from the training set
def display_sample_images(batch):
    images, labels = batch
    plt.figure(figsize=(12, 8))
    for i in range(5):
        plt.subplot(1, 5, i+1)
        plt.imshow(images[i])
        plt.axis('off')
    plt.show()

display_sample_images(next(train_dataset))


In [ ]:
# Display a sample image from the augmented training set
def display_augmented_images(batch):
    images, labels = batch
    plt.figure(figsize=(12, 8))
    for i in range(5):
        plt.subplot(1, 5, i+1)
        plt.imshow(images[i])
        plt.axis('off')
    plt.show()

In [ ]:
# Define ImageDataGenerators for training and validation
datagen_train_augment = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% of the data will be used for validation
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen_val_augment = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load the training and validation datasets
train_dataset_augment = datagen_train_augment.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

val_dataset_augment = datagen_val_augment.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)

display_sample_images(next(train_dataset_augment))

## **Model Definition and Training**

Without Augmentation

In [34]:
# This function makes our lives easier by helping us plot the training curves in one call
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [31]:
# Define the VGG16 model
def create_vgg16_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
input_shape = (224, 224, 3)
num_classes = len(train_dataset.class_indices)

# Compile and train the VGG16 model
vgg16_model = create_vgg16_model(input_shape, num_classes)
vgg16_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg16_model.summary()

In [ ]:
# Train the model
vgg16_model_history = vgg16_model.fit(train_dataset, epochs=10, validation_data=val_dataset)

In [35]:
# Define the ResNet50 model
def create_resnet50_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
# Compile and train the ResNet50 model
resnet50_model = create_resnet50_model(input_shape, num_classes)
resnet50_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet50_model.summary()

In [ ]:
# Train the model
resnet50_model_history = resnet50_model.fit(train_dataset, epochs=10, validation_data=val_dataset)

In [ ]:
plot_loss_curves(resnet50_model_history)

With Augmentation

In [ ]:
# Compile and train the VGG16 model with augmentation
vgg16_model_aug = create_vgg16_model(input_shape, num_classes)
vgg16_model_aug.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg16_model_aug.fit(train_dataset_augment, epochs=10, validation_data=val_dataset_augment)

In [ ]:
# Compile and train the ResNet50 model with augmentation
resnet50_model_aug = create_resnet50_model(input_shape, num_classes)
resnet50_model_aug.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet50_model_aug.fit(train_dataset_augment, epochs=10, validation_data=val_dataset_augment)